In [ ]:


import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  %pip install --quiet git+https://github.com/raultruco/cryptowatson-indicators#egg=cryptowatson-indicators
  %pip install --quiet git+https://github.com/mementum/backtrader.git@0fa63ef4a35dc53cc7320813f8b15480c8f85517#egg=backtrader

import backtrader as bt
from cryptowatsonindicators import datas, strategies
import requests
requests.packages.urllib3.disable_warnings()
import pprint
pprint = pprint.PrettyPrinter(indent=2, sort_dicts=False, compact=False).pprint   # pprint with defaults

%matplotlib inline

# Fear and Greed Simulator

Usage...

### Variables

In [ ]:
# simulator_mode = ['weighted'][0]  # select the mode index [x]
ticker_symbol = "BTCUSDT"         # currently only works with BTCUSDT
start_date = '01/01/2020'    # start date of the simulation
min_order_period = 5       # Number of days between purchases
weighted_buy_amount = 100             # Amount purchased in standard DCA
weight_type = ""             # TBD

### Run Simulator!

In [ ]:
cerebro = bt.Cerebro(stdstats=True)

# Add data feed: get ticker close prices time series from nasdaqdatalink  
ticker_data = datas.get_nasdaq_ticker_time_series(start_date=start_date)

ticker_data_feed = bt.feeds.PandasData(
    dataname=ticker_data,
    datetime=0,
    high=None,
    low=None,
    open=1,     # use column index 1 ('Value') as open price
    close=1,    # use column index 1 ('Value') as open price
    volume=None,
    openinterest=None,
)

cerebro.adddata(ticker_data_feed)

# Add strategy to test
cerebro.addstrategy(strategies.FngStrategy, weighted_buy_amount=weighted_buy_amount,
                min_order_period=min_order_period, weight_type=weight_type, log=False, debug=False)

# Add cash to the virtual broker
# cerebro.broker.setcash(100000.0)    # default: 10k

start_portfolio_value = cerebro.broker.getvalue()
cerebro.run()

end_portfolio_value = cerebro.broker.getvalue()
pnl_value = end_portfolio_value - start_portfolio_value
pnl_percent = (pnl_value / start_portfolio_value) * 100
pnl_sign = '' if pnl_value < 0 else '+'

# print("----------------------------------------")
print(f"{'Start value:':<12} {start_portfolio_value:2f} USD")
print(f"{'Final value:':<12} {end_portfolio_value:2f}  USD")
print(f"{'PnL:':<11} {pnl_sign}{pnl_value:.2f} USD ({pnl_sign}{pnl_percent:.2f}%)")

cerebro.plot(volume=False) # volume=False, iplot=True, style='bar', stdstats=False
